In [10]:
from twitter.search import Search
from twitter.scraper import Scraper
import pandas as pd

email, username, password = "b10705052@ntu.edu.tw", "yehyouming", "Ym911216"

search = Search(email, username, password, save=False, debug=1)
scraper = Scraper(email, username, password, debug=1, save=False)

res = search.run(
    limit=1,
    retries=1,
    queries=[{"category": "Top", "query": "ai"}],
)

# get the first tweet
r = res[0][0]

if r["content"]["itemContent"]["tweet_results"]["result"]["__typename"] == "Tweet":
    full_text = r["content"]["itemContent"]["tweet_results"]["result"]["legacy"][
        "full_text"
    ]
    rest_id = r["content"]["itemContent"]["tweet_results"]["result"]["rest_id"]
    print("tweet id:", rest_id)

    tweets_details = scraper.tweets_details([rest_id], count=100, limit=100)

    df_tweets_details = (
        pd.json_normalize(
            tweets_details,
            record_path=[
                "data",
                "threaded_conversation_with_injections_v2",
                "instructions",
            ],
        )["entries"]
        .dropna()
        .explode()
        .apply(pd.Series)["content"]
        .apply(pd.Series)["items"]
        .dropna()
        .explode()
        .apply(pd.Series)["item"]
        .apply(pd.Series)["itemContent"]
        .apply(pd.Series)
        .pipe(lambda df: df[df["__typename"] != "TimelineTimelineCursor"])[
            "tweet_results"
        ]
        .apply(pd.Series)["result"]
        .apply(pd.Series)
        .pipe(lambda df: df[df["__typename"] != "TweetWithVisibilityResults"])["legacy"]
        .apply(pd.Series)
        .pipe(lambda x: pd.concat([x, x["entities"].apply(pd.Series)], axis=1))
        .assign(created_at=lambda x: pd.to_datetime(x["created_at"]))
        .sort_values("created_at", ascending=False)
        .reset_index(drop=True)
        .drop("entities", axis=1)[
            [
                "created_at",
                "id_str",
                "user_id_str",
                "full_text",
                "favorite_count",
                "urls",
                "hashtags",
                "user_mentions",
            ]
        ]
    )


df_tweets_details

2024-04-17 23:59:41.856 [DEBUG] :: ai
2024-04-17 23:59:43.491 [DEBUG] :: [success] Returned 18 search results for ai
tweet id: 1780626667997659319


TweetDetail:   0%|          | 0/1 [00:00<?, ?it/s]

2024-04-17 23:59:45.017 [DEBUG] :: [200]
2024-04-17 23:59:45.017 [DEBUG] :: /i/api/graphql/zXaXQgfyR4GxE21uwYQSyA/TweetDetail
2024-04-17 23:59:45.018 [DEBUG] :: remaining: 148/150 requests
2024-04-17 23:59:45.018 [DEBUG] :: reset:     14.33 minutes


TweetDetail: 100%|██████████| 1/1 [00:01<00:00,  1.32s/it]
C:\Users\user\AppData\Local\Temp\ipykernel_20964\2723020308.py:53: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(created_at=lambda x: pd.to_datetime(x["created_at"]))


,created_at,id_str,user_id_str,full_text,favorite_count,urls,hashtags,user_mentions
0,2024-04-17 15:59:34+00:00,1780627171209257149,1656328215839776769,@NerisQueenAI Perfection every time 😍❤️,0,[],[],"[{'id_str': '1734980797755535360', 'name': 'Ne..."
1,2024-04-17 15:59:18+00:00,1780627102896963893,1663612280133795840,@NerisQueenAI You look so good Neris. You’re s...,0,[],[],"[{'id_str': '1734980797755535360', 'name': 'Ne..."
2,2024-04-17 15:59:15+00:00,1780627089953050798,1726640564064354304,@NerisQueenAI 💖💖💖😊😊🤩🤩❤️❤️😍😍😍💕💕🥰😘😘♥️♥️😘🥰🥰💕😍😍❤️💖...,0,[],[],"[{'id_str': '1734980797755535360', 'name': 'Ne..."
3,2024-04-17 15:57:56+00:00,1780626757848379830,1663612280133795840,@NerisQueenAI You’re awesome. 💗💗🔥🔥💖💕💕 https://...,0,[],[],"[{'id_str': '1734980797755535360', 'name': 'Ne..."
